In [8]:
import tensorflow as tf
import numpy as np
!pip install patool
!pip install pandas
import patoolib
import pandas as pd

In [9]:
patoolib.extract_archive('ChessEvals.zip')
dataframe = pd.read_csv('ChessEvals/chessData.csv')

INFO patool: Extracting ChessEvals.zip ...
INFO:patool:Extracting ChessEvals.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_77nk5sw_ -- ChessEvals.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_77nk5sw_ -- ChessEvals.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... ChessEvals.zip extracted to `ChessEvals3' (multiple files in root).
INFO:patool:... ChessEvals.zip extracted to `ChessEvals3' (multiple files in root).


In [ ]:
input = [np.zeros((100, 8, 8)) for i in range (100)]
y = np.random.rand(100)
y

array([7.99878778e-01, 5.60724052e-01, 3.53550835e-01, 4.70149441e-01,
       7.93751793e-01, 1.83540543e-01, 7.65759733e-01, 9.66057009e-01,
       6.80285053e-01, 3.37507989e-02, 6.96265849e-01, 9.79748300e-01,
       6.46672949e-01, 2.61400342e-01, 9.91722669e-01, 8.58807439e-01,
       5.78260150e-04, 9.33339233e-01, 6.89877986e-01, 9.06677592e-01,
       1.72728683e-01, 4.30171098e-01, 6.52608912e-01, 7.04302358e-02,
       7.01299324e-01, 5.59666121e-01, 2.99560000e-01, 3.56195441e-01,
       7.43210216e-01, 1.22912989e-01, 9.03216989e-01, 4.23477697e-01,
       3.47356792e-01, 4.43238648e-01, 8.63886291e-01, 2.32681226e-01,
       8.79970322e-01, 5.68398181e-01, 9.95150364e-01, 6.08867034e-02,
       4.74465991e-01, 3.60760273e-01, 7.82625058e-01, 1.44001335e-01,
       3.07847514e-01, 2.49606093e-01, 8.22874117e-01, 1.15240810e-02,
       2.22328509e-01, 4.18942403e-01, 1.37926697e-01, 7.04591180e-01,
       9.54955179e-01, 6.85285108e-01, 8.16237634e-01, 7.41232577e-01,
      

In [ ]:
x1 = tf.keras.layers.Input((14, 8, 8))
x2 = tf.keras.layers.Conv2D (16, (3,3), activation = 'relu', data_format = 'channels_first') (x1)
x3 = tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', data_format = 'channels_first') (x2)
x4 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', data_format = 'channels_first') (x3)
x5 = tf.keras.layers.Flatten()(x4)
x6 = tf.keras.layers.Dense(64, activation = 'relu')(x5)
x7 = tf.keras.layers.Dense(32, activation = 'relu')(x6)
x8 = tf.keras.layers.Dense(1 )(x7)
model = tf.keras.models.Model (inputs = x1, outputs = x8)


In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 6, 6)          2032      
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 4, 4)          4640      
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 2, 2)          4624      
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080

In [ ]:
model.compile(loss ='mse', optimizer = 'adam', accuracy = ['mse'])

NameError: name 'model' is not defined

In [ ]:
model.fit(input, y)

In [ ]:
fen = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [12]:

'''
Matric representation order: black, white:  king, queen, rook, knight, bishop, pawn
'''
def bitMap(fen):
  board = chess.Board(fen)
  orderMapping = {
    'k': 0,
    'K': 6,
    'q': 1,
    'Q': 7,
    'r': 2,
    'R': 8,
    'n': 3,
    'N': 9,
    'b': 4,
    'B': 10,
    'p': 5,
    'P': 11
  }
  material = {
    'k': 0,
    'K': 0,
    'q': -9,
    'Q': 9,
    'r': -5,
    'R': 5,
    'n': -3,
    'N': 3,
    'b': -3,
    'B': 3,
    'p': -1,
    'P': 1
  }
  alphabet = 'abcdefgh'
  bitMap = np.zeros((14, 8, 8))
  positions, turn, castlingRights, enPeasant, half1, half2 = fen.split(' ')
  row, column = [0, 0]
  whiteMaterial, blackMaterial = 0, 0
  for letter in positions:
    if letter.isnumeric():
      column += int(letter)
    elif letter == '/':
      row += 1
      column = 0
    else:
      peiceIndex = orderMapping[letter]
      if letter.isupper():
        bitMap[peiceIndex, row, column] = 1
      else:
        bitMap[peiceIndex, row, column] = -1
      if letter.isupper():
        whiteMaterial += material[letter]
      else:
        blackMaterial += material[letter]
      column += 1

  #showing the model the possible squares we can move to, which is known to help the model a lot
  first, second = 12, 13
  if not board.turn:
    first, second = 13, 12

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[first, row, column] = 1
  board.turn = not board.turn #looking at the possible moves for the person whos turn its not

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[second, row, column] = 1
  #since we originally placed the pieces by defining rank 0 as the eigth index, we have to flip this list to stay in accordance to the definition
  bitMap[12] = bitMap[12][::-1]
  bitMap[13] = bitMap[13][::-1]
  return bitMap, whiteMaterial, blackMaterial


#trying a different board representation to see if it performs better (after testing, it does. The current model uses this function as opposed to bitMap(fen).)
def compressedBitMap(fen):
  board = chess.Board(fen)
  orderMapping = {
    'k': 0,
    'K': 0,
    'q': 1,
    'Q': 1,
    'r': 2,
    'R': 2,
    'n': 3,
    'N': 3,
    'b': 4,
    'B': 4,
    'p': 5,
    'P': 5
  }
  material = {
    'k': 0,
    'K': 0,
    'q': -9,
    'Q': 9,
    'r': -5,
    'R': 5,
    'n': -3,
    'N': 3,
    'b': -3,
    'B': 3,
    'p': -1,
    'P': 1
  }
  alphabet = 'abcdefgh'
  bitMap = np.zeros((8, 8, 8))
  positions, turn, castlingRights, enPeasant, half1, half2 = fen.split(' ')
  row, column = [0, 0]
  whiteMaterial, blackMaterial = 0, 0
  for letter in positions:
    if letter.isnumeric():
      column += int(letter)
    elif letter == '/':
      row += 1
      column = 0
    else:
      peiceIndex = orderMapping[letter]
      if letter.isupper():
        bitMap[peiceIndex, row, column] = 1
      else:
        bitMap[peiceIndex, row, column] = -1
      if letter.isupper():
        whiteMaterial += material[letter]
      else:
        blackMaterial += material[letter]
      column += 1

  #showing the model the possible squares we can move to, which is known to help the model a lot
  first, second = 6, 7
  if not board.turn:
    first, second = 7, 6

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[first, row, column] = 1
  board.turn = not board.turn #looking at the possible moves for the person whos turn its not

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[second, row, column] = 1
  #since we originally placed the pieces by defining rank 0 as the eigth index, we have to flip this list to stay in accordance to the definition
  bitMap[6] = bitMap[6][::-1]
  bitMap[7] = bitMap[7][::-1]
  return bitMap, whiteMaterial, blackMaterial
